In [222]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import os

In [168]:
import torchvision.models as models

In [175]:
transforms = transforms.transforms.Compose([transforms.Resize([224,224]),transforms.ToTensor()])

In [190]:
list(models.resnet34().children())

[Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False),
 BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 ReLU(inplace=True),
 MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False),
 Sequential(
   (0): BasicBlock(
     (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (relu): ReLU(inplace=True)
     (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   )
   (1): BasicBlock(
     (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (relu): ReLU(inplace=True)
     (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), pad

In [282]:
data_dir='/Users/paulbruffett/code/ML/invasive/train'

image_datasets = datasets.ImageFolder(data_dir,transform=transforms)
train_set, val_set = torch.utils.data.random_split(image_datasets, [int(len(image_datasets)*0.8), int(len(image_datasets)*0.2)])
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)
valid_loader = torch.utils.data.DataLoader(val_set, batch_size=64, shuffle=False)

In [177]:
image_datasets.classes

['invasive', 'not invasive']

In [178]:
"""import pandas as pd
import shutil
labels = pd.read_csv(data_dir+"/train_labels.csv")"""

'import pandas as pd\nimport shutil\nlabels = pd.read_csv(data_dir+"/train_labels.csv")'

In [179]:
"""for i in labels.iterrows():
    file_name = str(i[1]['name'])+'.jpg'
    if i[1]['invasive'] == 1:
        shutil.move(data_dir+"/train/"+file_name,data_dir+"/train/"+"invasive/"+file_name)
    else:
        shutil.move(data_dir+"/train/"+file_name,data_dir+"/train/"+"not-invasive/"+file_name)
    print(i[1]['name'],i[1]['invasive'])
"""

'for i in labels.iterrows():\n    file_name = str(i[1][\'name\'])+\'.jpg\'\n    if i[1][\'invasive\'] == 1:\n        shutil.move(data_dir+"/train/"+file_name,data_dir+"/train/"+"invasive/"+file_name)\n    else:\n        shutil.move(data_dir+"/train/"+file_name,data_dir+"/train/"+"not-invasive/"+file_name)\n    print(i[1][\'name\'],i[1][\'invasive\'])\n'

In [180]:
def create_body():
    "Cut off the body of a typically pretrained `model` at `cut` (int) or cut the model as specified by `cut(model)` (function)."
    model = models.resnet18(pretrained=True)
    return nn.Sequential(*list(model.children())[:-1])


In [257]:
class conv_net(nn.Module):
    def __init__(self):
        super().__init__()

        self.body = create_body()

        conv_out_size = self._get_conv_out([3,224,224])
        self.fc = nn.Sequential(
            nn.Linear(conv_out_size, 512),
            nn.ReLU(),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )

    def _get_conv_out(self, shape):
        o = self.body(torch.zeros(1, *shape))
        return int(np.prod(o.size()))

    def forward(self, x):
        conv_out = self.body(x).view(x.size()[0], -1)
        return self.fc(conv_out)

In [258]:
model = conv_net()

In [259]:
list(model.children())

[Sequential(
   (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
   (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (2): ReLU(inplace=True)
   (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
   (4): Sequential(
     (0): BasicBlock(
       (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu): ReLU(inplace=True)
       (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     )
     (1): BasicBlock(
       (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu): ReLU(inplace=True)
 

In [260]:
n_layers = len(list(model.parameters()))-6
for layer in model.parameters():
    layer.requires_grad=False
    n_layers -=1
    if n_layers==0:
        break

In [261]:
for l in model.parameters():
    print(l.requires_grad)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True
True
True
True
True


In [262]:
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [263]:
epochs = 5
print_every = 200
for e in range(epochs):
    running_loss = 0
    for step, (x,y) in enumerate(trainloader):
        # Flatten MNIST images into a 784 long vector
#        x.resize_(x.size()[0], 784)
        y=y.reshape(64,-1).float()
        
        optimizer.zero_grad()
        
        # Forward and backward passes
        output = model.forward(x)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        

        if step % print_every == 0:
          print("Epoch: {}/{}... ".format(e+1, epochs),
                "Loss: {:.4f}".format(running_loss/print_every))

          running_loss = 0

Epoch: 1/5...  Loss: 0.0035


KeyboardInterrupt: 

In [275]:
len(trainloader)

36

In [274]:
train_set, val_set = torch.utils.data.random_split(trainloader, [len(trainloader.dataset), 0])

ValueError: Sum of input lengths does not equal the length of the input dataset!

In [270]:
2295*0.8

1836.0

In [268]:
2295*0.2

459.0